In [0]:
%sql
-- Aggregate company profit KPIs from orders data: calculates total revenue, total cost, profit, and profit margin.
CREATE OR REPLACE TABLE supply_dev.gold.company_profit_kpi AS
SELECT
  SUM(revenue_generated)                                    AS total_revenue,
  SUM(manufacturing_costs + shipping_costs)                 AS total_cost,
  SUM(revenue_generated) 
    - SUM(manufacturing_costs + shipping_costs)             AS profit,
  (SUM(revenue_generated) 
    - SUM(manufacturing_costs + shipping_costs))
    / SUM(revenue_generated)                                AS profit_margin
FROM supply_dev.silver.orders;

In [0]:
%sql
SELECT *
FROM supply_dev.gold.company_profit_kpi;


In [0]:
%sql
CREATE or REPLACE TABLE supply_dev.gold.product_profit_kpi AS
WITH base AS (
  SELECT
    product_type,
    SUM(revenue_generated) AS total_revenue,
    SUM(manufacturing_costs + Shipping_costs) AS total_cost
  FROM supply_dev.silver.orders
  GROUP BY product_type
)
SELECT
  product_type,
  total_revenue,
  total_cost,
  total_revenue - total_cost AS profit,
  (total_revenue - total_cost) / total_revenue AS profit_margin
FROM base
ORDER BY profit_margin DESC;

In [0]:
%sql
SELECT *
FROM supply_dev.gold.product_profit_kpi

In [0]:
%sql
CREATE OR REPLACE TABLE supply_dev.gold.inventory_efficient_kpi AS
WITH base AS(
  SELECT
  product_type,
  SUM(stock_levels * manufacturing_costs) AS inventory_value,
  SUM(revenue_generated) AS sales_value
  FROM supply_dev.silver.orders
  GROUP BY product_type

)
SELECT
  product_type,
  inventory_value,
  sales_value,
  inventory_value / sales_value AS inventory_efficiency
FROM base
ORDER BY inventory_efficiency DESC;
    

In [0]:
%sql
SELECT *
FROM supply_dev.gold.inventory_efficient_kpi;

In [0]:
%sql
CREATE OR REPLACE TABLE supply_dev.gold.shipping_cost_impact_kpi AS
WITH base AS (
  SELECT
    SUM(revenue_generated) AS total_revenue,
    SUM(shipping_costs) AS total_shipping_costs
  FROM supply_dev.silver.orders
)
SELECT
  total_revenue,
  total_shipping_costs, 
  total_shipping_costs / total_revenue AS shipping_cost_impact
FROM base;

In [0]:
%sql
CREATE OR REPLACE TABLE supply_dev.gold.product_shipping_cost_kpi AS
WITH base AS (
  SELECT
    product_type,
    SUM(revenue_generated) AS total_revenue,
    SUM(shipping_costs) AS total_shipping_cost
  FROM supply_dev.silver.orders
  GROUP BY product_type
)
SELECT
  product_type,
  total_revenue,
  total_shipping_cost,
  total_shipping_cost / total_revenue AS shipping_cost_ratio
FROM base
ORDER BY shipping_cost_ratio DESC;


In [0]:
%sql
SELECT *
FROM supply_dev.gold.shipping_cost_impact_kpi;


In [0]:
%sql
SELECT *
FROM supply_dev.gold.product_shipping_cost_kpi;


In [0]:
%sql
CREATE OR REPLACE TABLE supply_dev.gold.on_time_delivery_kpi AS
WITH base AS (
  SELECT
    CASE 
      WHEN shipping_times <= lead_time THEN 1
      ELSE 0
    END AS on_time_flag
  FROM supply_dev.silver.orders
)
SELECT
  AVG(on_time_flag) AS on_time_delivery_rate
FROM base;


In [0]:
%sql
CREATE OR REPLACE TABLE supply_dev.gold.product_on_time_delivery_kpi AS
WITH base AS (
  SELECT
    product_type,
    CASE 
      WHEN shipping_times <= lead_time THEN 1
      ELSE 0
    END AS on_time_flag
  FROM supply_dev.silver.orders
)
SELECT
  product_type,
  AVG(on_time_flag) AS on_time_delivery_rate
FROM base
GROUP BY product_type
ORDER BY on_time_delivery_rate ASC;


In [0]:
%sql
SELECT *
FROM supply_dev.gold.on_time_delivery_kpi;


In [0]:
%sql
SELECT *
FROM supply_dev.gold.product_on_time_delivery_kpi;
